In [1]:
from grandma_stock_valuation import FileLogger, YahooDataLoader, batchValuation, addCashPortfolio, getCorrelationWeight, allocatePortfolio
import time
import pandas as pd

logger = FileLogger()
logPrint = logger.logPandas


### Refresh data of each instrument

In [2]:
LOAD_SAVED_DATA = True

d_instrument = {
    'IVV':'SP500',
    '3073.HK':'Greater China',
    'VPL':'Developed Asia-Pacific',
    'IEV':'Europe',
    'ASEA':'SE Asia',

    'XT':'Global Tech',
    'IXG':'Global Finance',
    'IXJ':'Global Healthcare',
    'KXI':'Global Consumer Staple',
    'RXI':'Global Consumer Discretionary',

    'AAXJ':'Asia ex Japan',
    'EZU':'Eurozone'
}

d_instrument_data = {}
for ticker, name in d_instrument.items():
    logPrint(f"{ticker}: {name}")

    if LOAD_SAVED_DATA:
        df = pd.read_csv(f'_data/{ticker}_EOD.csv.gz')
        df['date'] = pd.to_datetime(df['date'])
    else:
        yahoo = YahooDataLoader(ticker, verbose=0, printfunc=logPrint)
        df = yahoo.queryEOD(save=True)

    d_instrument_data[ticker] = df
    time.sleep(1)



2022-02-28 23:13:29,055 INFO IVV: SP500
2022-02-28 23:13:30,082 INFO 3073.HK: Greater China
2022-02-28 23:13:31,092 INFO VPL: Developed Asia-Pacific
2022-02-28 23:13:32,105 INFO IEV: Europe
2022-02-28 23:13:33,128 INFO ASEA: SE Asia
2022-02-28 23:13:34,146 INFO XT: Global Tech
2022-02-28 23:13:35,161 INFO IXG: Global Finance
2022-02-28 23:13:36,189 INFO IXJ: Global Healthcare
2022-02-28 23:13:37,217 INFO KXI: Global Consumer Staple
2022-02-28 23:13:38,239 INFO RXI: Global Consumer Discretionary
2022-02-28 23:13:39,264 INFO AAXJ: Asia ex Japan
2022-02-28 23:13:40,282 INFO EZU: Eurozone


### Valuate the instruments

In [3]:
df_valuation_metrics, d_fig = batchValuation(d_instrument_data, verbose=0, printfunc=logPrint)

df_valuation_metrics

,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years
0,IVV,0.971075,10.00274,0.134429,439.609985,411.677817,0.06785,0.504725
1,3073.HK,0.916861,10.0,0.082741,48.360001,52.484986,-0.078594,-0.650293
2,VPL,0.890418,10.00274,0.067409,74.730003,77.180285,-0.031748,-0.214008
3,IEV,0.781494,10.00274,0.052764,51.27,49.239752,0.041232,0.78144
4,ASEA,0.347906,10.00274,0.016536,15.7,14.361431,0.093206,5.636511
5,XT,0.951682,6.931507,0.186458,57.650002,64.250108,-0.102725,-1.915393
6,IXG,0.889712,10.00274,0.079571,80.639999,77.375727,0.042187,0.530187
7,IXJ,0.960838,10.00274,0.110765,83.989998,82.926623,0.012823,0.115769
8,KXI,0.961954,10.00274,0.071416,63.009998,60.95388,0.033732,0.472333
9,RXI,0.948823,10.00274,0.118697,156.380005,163.957313,-0.046215,-0.548561


### Construct a portfolio

In [13]:
instruments_select = ['IVV', '3073.HK', 'VPL', 'IEV', 'ASEA']

df_portfolio = df_valuation_metrics[df_valuation_metrics['ticker'].isin(instruments_select)]
df_portfolio = addCashPortfolio(df_portfolio)

d_instrument_prices = {ticker:data for ticker, data in d_instrument_data.items() if ticker in instruments_select}
d_weights = getCorrelationWeight(d_instrument_prices, with_cash=True, cash_name='cash', verbose=2)
df_portfolio['weight'] = df_portfolio['ticker'].apply(lambda t: d_weights[t] )

df_portfolio['portfolio'] = allocatePortfolio(df_portfolio['over_value_years'], weights=df_portfolio['weight'])
df_portfolio

IVV: Selected 2518 rows over 2518 dates from 2012-02-27 to 2022-02-25.
3073.HK: Selected 2465 rows over 2465 dates from 2012-02-27 to 2022-02-24.
VPL: Selected 2518 rows over 2518 dates from 2012-02-27 to 2022-02-25.
IEV: Selected 2518 rows over 2518 dates from 2012-02-27 to 2022-02-25.
ASEA: Selected 2518 rows over 2518 dates from 2012-02-27 to 2022-02-25.


,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years,weight,portfolio
0,IVV,0.971075,10.00274,0.134429,439.609985,411.677817,0.06785,0.504725,0.140523,0.079218
1,3073.HK,0.916861,10.0,0.082741,48.360001,52.484986,-0.078594,-0.650293,0.121622,0.470024
2,VPL,0.890418,10.00274,0.067409,74.730003,77.180285,-0.031748,-0.214008,0.101613,0.189786
3,IEV,0.781494,10.00274,0.052764,51.27,49.239752,0.041232,0.781440,0.121065,0.043033
4,ASEA,0.347906,10.00274,0.016536,15.7,14.361431,0.093206,5.636511,0.348510,0.000038
5,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.166667,0.217901


In [14]:
instruments_select = ['IVV', '3073.HK', 'VPL', 'IEV', 'ASEA']

df_portfolio = df_valuation_metrics[df_valuation_metrics['ticker'].isin(instruments_select)]
df_portfolio = addCashPortfolio(df_portfolio)

d_instrument_prices = {ticker:data for ticker, data in d_instrument_data.items() if ticker in instruments_select}
d_weights = getCorrelationWeight(d_instrument_prices, with_cash=True, cash_name='cash', verbose=2)
df_portfolio['weight'] = df_portfolio['ticker'].apply(lambda t: d_weights[t] )

df_portfolio['portfolio'] = allocatePortfolio(df_portfolio['over_value_years'], scale=1, weights=df_portfolio['weight'])
df_portfolio

IVV: Selected 2518 rows over 2518 dates from 2012-02-27 to 2022-02-25.
3073.HK: Selected 2465 rows over 2465 dates from 2012-02-27 to 2022-02-24.
VPL: Selected 2518 rows over 2518 dates from 2012-02-27 to 2022-02-25.
IEV: Selected 2518 rows over 2518 dates from 2012-02-27 to 2022-02-25.
ASEA: Selected 2518 rows over 2518 dates from 2012-02-27 to 2022-02-25.


,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years,weight,portfolio
0,IVV,0.971075,10.00274,0.134429,439.609985,411.677817,0.06785,0.504725,0.140523,0.127170
1,3073.HK,0.916861,10.0,0.082741,48.360001,52.484986,-0.078594,-0.650293,0.121622,0.349355
2,VPL,0.890418,10.00274,0.067409,74.730003,77.180285,-0.031748,-0.214008,0.101613,0.188681
3,IEV,0.781494,10.00274,0.052764,51.27,49.239752,0.041232,0.781440,0.121065,0.083077
4,ASEA,0.347906,10.00274,0.016536,15.7,14.361431,0.093206,5.636511,0.348510,0.001863
5,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.166667,0.249854
